In [2]:
%run 01-model.ipynb

'' is not a valid SBML 'SId'.


Do we even have any cofactors?

PsiD: pyridoxal-5′-phosphate (PLP) i planter, men hvad med svampe?

PSIH: nej, ikke så vidt jeg kan finde ud af
cp450 reductase som har cofaktorerne (FMN, FAD and NADPH)

PsiK: ATP 
PsiMx2: S-adenosyl-L-methionine (SAM) 

